In [2]:
import numpy as np
import pandas as pd

In [27]:
df = pd.read_csv('UH_RL_rats.csv')

In [28]:
df = df.drop(df.columns[[range(24)]],axis=1)
subjects = df['subject'].unique()

/Users/erika/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/base.py:1700: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)


In [30]:
subjects.dtype

dtype('int64')

In [31]:
df.head()

,session,subject,lever,response,feedback,state,action,reward,alpha_gain,alpha_loss,beta,Q,PE
0,1,10,A,rich,0.0,1.0,1.0,0.0,0.139603,1.0,20.0,0.000000,-0.500000
1,1,10,A,rich,1.0,1.0,1.0,1.0,0.139603,1.0,20.0,0.139603,1.000000
2,1,10,B,lean,0.0,1.0,2.0,0.0,0.139603,1.0,20.0,0.000000,-0.500000
3,1,10,A,rich,1.0,1.0,1.0,1.0,0.139603,1.0,20.0,0.259718,0.860397
4,1,10,B,lean,0.0,1.0,2.0,0.0,0.139603,1.0,20.0,0.000000,0.000000


In [22]:
# check state
df['state'].unique()

array([1.])

In [37]:
# divide df into 5 dfs according to session
# '1,11,16,6,Best'
session = df['session'].unique()
df_1 = df[df['session']=='1']
df_11 = df[df['session']=='11']
df_16 = df[df['session']=='16']
df_6 = df[df['session']=='6']
df_best = df[df['session']=='Best']
df_list = [df_1,df_11,df_16,df_6,df_best]

In [44]:
# action list, reward list, and Q list
action_list = []
reward_list = []
Q_list = []

# the k-th array in each list corresponds to (1+k%20)-th object in session[int(k/5)]
for i in range(len(session)):
    for j in range(len(subjects)):
        subj = j + 1 # subjects 1-20
        df_sess = df_list[i]
        action = np.array(df_sess['action'][df_sess['subject']==subj])
        reward = np.array(df_sess['reward'][df_sess['subject']==subj])
        Q = np.array(df_sess['Q'][df_sess['subject']==subj])
                
        action_list.append(action)
        reward_list.append(reward)
        Q_list.append(Q)



In [ ]:
# to write
with open('action_list', 'w') as f:
    for df in action_list:
        f.write(df + '\n')

with open('reward_list', 'w') as f:
    for df in reward_list:
        f.write(df + '\n')

with open('Q_list', 'w') as f:
    for df in Q_list:
        f.write(df + '\n')

In [ ]:
# to read
with open(the_filename, 'r') as f:
    my_list = [line.rstrip('\n') for line in f]